# Определение региона для бурения новой скважины по прогнозируемой прибыли

### Описание проекта

Нефтедобывающей компании необходимо определить место бурения новой скважины.

**Цели и задачи**
* В трёх нефтедобывающих регионах (по 10000 скважин в каждом) взяты пробы нефти, измерено качество сырья объём её запасов.
* Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.
* Проанализировать возможную прибыль и риски техникой Bootstrap.

**Описание данных**

Данные геологоразведки трёх регионов находятся в файлах: `geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv`.

* `id` — уникальный идентификатор скважины;
* `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**
* Использовать модель линейной регрессии (остальные — недостаточно предсказуемые).
* Исследовать 500 точек при геологоразведке, выбрать 200 с наибольшими запасами для расчёта прибыли.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* Стоимость одного барреля нефти 450 рублей. Объём указан в 1000 баррелей.
* Выбрать регионы, в которых вероятность убытков меньше 2.5%. Среди регионов, прошедших оценку рисков, выбрать один с наибольшей средней прибылью.

### Структура проекта

1. Загрузка и подготовка данных
2. Обучение и проверка модели
3. Подготовка к расчёту прибыли
4. Расчёт прибыли и рисков
    * 4.1 Расчет прибыли
    * 4.2 Расчёт рисков
5. Общий вывод

## 1. Загрузка и подготовка данных

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import os

from numpy.random import RandomState
from scipy import stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# открываем файлы
dataset_names = ['geo_data_0.csv', 'geo_data_1.csv', 'geo_data_2.csv']
data = []

for df in dataset_names:
    if os.path.exists(f'C:\\Users\\Anna\\Desktop\\Python\\Yandex Praktikum\\Datasets\\project_8\\{df}'):
        data.append(pd.read_csv(f'C:\\Users\\Anna\\Desktop\\Python\\Yandex Praktikum\\Datasets\\project_8\\{df}'))
    elif os.path.exists(f'/datasets/{df}'):
        data.append(pd.read_csv(f'/datasets/{df}'))
    else:
        print(f'Something is wrong with {df}')

Выведем первые 5 строк датасетов и посмотрим на общую информацию.

In [3]:
# функция для выведения первых 5 строк датасета и обшей информации
def data_info(data):
    display(data.head())
    data.info()
    print()
    print(f'Количество явных дубликатов:{data.duplicated().sum()}')

In [4]:
data_info(data[0])

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество явных дубликатов:0


In [5]:
data_info(data[1])

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество явных дубликатов:0


In [6]:
data_info(data[2])

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество явных дубликатов:0


* Данные соответствуют описанию. Пропусков и явных дубликатов нет.
* Типы данных соответствуют описанию.

Проверим наличие дублирующих id скважин.

In [7]:
data[0]['id'].value_counts()

fiKDv    2
QcMuo    2
bsk9y    2
A5aEY    2
bxg6G    2
        ..
xbq8v    1
jnyou    1
jnDQt    1
1KqaN    1
Gp52F    1
Name: id, Length: 99990, dtype: int64

In [8]:
data[0].loc[data[0]['id'] == 'fiKDv']

,id,f0,f1,f2,product
16633,fiKDv,0.157341,1.028359,5.585586,95.817889
90815,fiKDv,0.049883,0.841313,6.394613,137.346586


К сожалению, мы не знаем причин появления нескольких строк данных по одним и тем же скважинам и не знаем, можем ли мы объединить признаки эти скважин, поэтому избавимся от данных по этим скважинам.

In [9]:
#функцтя для удаления наблюдений с дублирующимися id
def remove_double (data):
    remove_list = data['id'].value_counts()[data['id'].value_counts() > 1].index
    data.query('id not in @remove_list', inplace=True)
    data.reset_index(inplace=True, drop=True)
    print(f'Осталось строк после удаления: {data.shape[0]}')

In [10]:
for df in data:
    remove_double(df)

Осталось строк после удаления: 99980
Осталось строк после удаления: 99992
Осталось строк после удаления: 99992


**Вывод**
* Данные успешно загружены и проанализированы. Данные соответствуют описанию. Пропусков и явных дубликатов нет. Типы данных соответствуют описанию.
* Обнаружены дублирующие id скважин, так как нет дополнительной информации, эти данные были удалены без значительного ущерба (20, 8 и 8 строк).

## 2. Обучение и проверка модели
Для каждого региона:

* Разделим данные на обучающую и валидационную выборкив соотношении 75:25.
* Обучим модель линейной регрессии по техническому заданию.
* Выведем средний запас предсказанного сырья и RMSE модели.

In [11]:
features_train = []
features_valid = []
target_train = []
target_valid = []

for df in data:
    features = df[['f0', 'f1', 'f2']]
    target = df['product']

    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=1505)
    features_train.append(X_train)
    features_valid.append(X_valid)
    target_train.append(y_train)
    target_valid.append(y_valid)

In [12]:
models = []
predictions = []
rmse = []

for i in range(len(data)):
    models.append(LinearRegression(n_jobs=-1))
    models[i].fit(features_train[i], target_train[i])
    predicted_values = models[i].predict(features_valid[i])
    predictions.append(pd.Series(data=predicted_values, index=target_valid[i].index))
    rmse.append(mean_squared_error(target_valid[i], predictions[i], squared=False))
    print(f'=== Регион {i} ===\n\tСредний запас предсказанного сырья: {predictions[i].mean():.2f}\n\tRMSE: {rmse[i]:.3f}\n')

=== Регион 0 ===
	Средний запас предсказанного сырья: 92.35
	RMSE: 37.980

=== Регион 1 ===
	Средний запас предсказанного сырья: 68.68
	RMSE: 0.896

=== Регион 2 ===
	Средний запас предсказанного сырья: 94.90
	RMSE: 40.163



**Выводы**
* В регионах 0 и 2 достаточно высокий средний показатель запаса предсказанного сырья (92.4 и 94.9 соответственно). Однако RMSE для данных регионов также высок (37.9 и 40.1 соответственно). Это свидетельствует о неоднозначности показателя, неточности модели регрессии.
* В регионе 1 средний показатель запаса предсказанного сырья составляет 68.7 (ниже остальных регионов). Однако RMSE в данном регионе также мал (0.9). Это говорит о точности предсказаний и качестве построенной модели.

## 3. Подготовка к расчёту прибыли

* Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.
* Сравним полученный объём сырья со средним запасом в каждом регионе.


* **min_product = budget / (top_wells * unit_income)**
    * budget - Бюджет на разработку скважин в регионе — 10 млрд рублей
    * top_wells, samp_wells - При расчёте выбирают 200 лучших скважин из 500 в выборке
    * unit_income - Доход с каждой единицы продукта составляет 450 тыс. рублей

In [13]:
BUDGET = 10000000000
TOP_WELLS = 200
SAMP_WELLS = 500
UNIT_INCOME = 450000

In [14]:
min_product = BUDGET / (TOP_WELLS * UNIT_INCOME)
print('Минимальный объём продукта с одной скважины для безубыточной разработки: {:.2f}'.format(min_product))

Минимальный объём продукта с одной скважины для безубыточной разработки: 111.11


**Выводы**

* Минимальный объём продукта для безубыточной разработки составляет **111.11**. Данный показатель рассчитан при условии выбора 200 лучших скважин из 500 в выборке.
* Данное значение превышает средние значения запасов скважин по всем регионам.

## 4. Расчёт прибыли и рисков
### 4.1 Расчёт прибыли

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

* Выберем скважины с максимальными значениями предсказаний и отберём 200 с максимальным показателем объёма сырья.
* Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
* Рассчитаем прибыль для полученного объёма сырья.

In [15]:
def profit(target, predictions, show_res=False):
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:TOP_WELLS]
    
    if show_res:
        print(f'\tСредний запас сырья среди скважин с максимальным показателем: {selected.mean():.2f}')
        print(f'\tСуммарный целевой объём сырья: {selected.sum():.2f}')
        print(f'\tПрибыль для полученного объёма сырья: {selected.sum() * UNIT_INCOME - BUDGET:,.2f}')
        
    return selected.sum() * UNIT_INCOME - BUDGET

In [16]:
for i in range(len(data)):
    print(f'=== Регион {i} ===')
    profit(target_valid[i], predictions[i], show_res=True)
    print()

=== Регион 0 ===
	Средний запас сырья среди скважин с максимальным показателем: 145.83
	Суммарный целевой объём сырья: 29166.69
	Прибыль для полученного объёма сырья: 3,125,010,478.94

=== Регион 1 ===
	Средний запас сырья среди скважин с максимальным показателем: 137.95
	Суммарный целевой объём сырья: 27589.08
	Прибыль для полученного объёма сырья: 2,415,086,696.68

=== Регион 2 ===
	Средний запас сырья среди скважин с максимальным показателем: 137.37
	Суммарный целевой объём сырья: 27474.62
	Прибыль для полученного объёма сырья: 2,363,580,135.90



### 4.2 Расчёт рисков


* Применим технику `Bootstrap` с 1000 выборок, чтобы найти распределение прибыли. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины. В нашем случае выберем случайно 500 точек и отберём 200 с максимальным показателем объёма сырья.
* Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [17]:
state = RandomState(1505) 
earnings = []
confidence_intervals =[]
risk = []

for i in range(len(data)):
    values = []
    confidence_interval = {}
    
    for j in range(1000):
        target_subsample = target_valid[i].sample(n=SAMP_WELLS, replace=True, random_state=state)
        preds_subsample = predictions[i][target_subsample.index]
        
        values.append(profit(target_subsample, preds_subsample))
        
    values = pd.Series(values)
    earnings.append(values)
    
    confidence_interval['lower'] = values.quantile(0.025)
    confidence_interval['upper'] = values.quantile(0.975)
#     confidence_interval = st.t.interval(alpha=0.95, df=len(values)-1, loc=values.mean(), scale=values.sem())
    confidence_intervals.append(confidence_interval)
    
    risk.append((earnings[i] < 0).mean())

In [18]:
for i in range(len(data)):
    print(f'=== Регион {i} ===')
    print(f'\tСредняя прибыль: {earnings[i].mean():,.2f}₽')
    print(f'\tДоверительный интервал: от {confidence_intervals[i]["lower"]:,.2f}₽ до {confidence_intervals[i]["upper"]:,.2f}₽')
#     print(f'\tДоверительный интервал: от {confidence_intervals[i][0]:,.2f}₽ до {confidence_intervals[i][1]:,.2f}₽')
    print(f'\tВероятность убытков: {risk[i]:.2%}')
    print()

=== Регион 0 ===
	Средняя прибыль: 472,462,346.48₽
	Доверительный интервал: от -45,817,238.22₽ до 1,003,195,711.77₽
	Вероятность убытков: 3.70%

=== Регион 1 ===
	Средняя прибыль: 511,539,237.12₽
	Доверительный интервал: от 113,008,671.98₽ до 938,622,822.28₽
	Вероятность убытков: 0.80%

=== Регион 2 ===
	Средняя прибыль: 375,966,071.42₽
	Доверительный интервал: от -164,657,624.97₽ до 912,258,774.31₽
	Вероятность убытков: 8.60%



**Выводы**

* При расчёте показателей для 200 наилучших скважин из 500 в выборке:
    * Средний запас сырья с одной скважины среди всех регионов (145.83, 137.95, 137.37 соответственно) превосходит минимально необходимый объём 111.11.

* При применении техники `Bootstrap`:
    * Оценка средней прибыли максимальна для региона 1 (511.54 млн.).
    * Только Регион 1 по уровню убытков (0.80%) не превосходит заданную границу по техническому заданию (2.5%).
    * Рекомендуем регион 1 для разработки.

## 5. Общий Вывод

* При начальном построении модели и предсказаний:
    * В регионах 0 и 2 достаточно высокий средний показатель запаса предсказанного сырья (92.4 и 94.9 соответственно). Однако RMSE для данных регионов также высок (**37.9** и **40.1** соответственно). Это свидетельствует о неоднозначности показателя, неточности модели регрессии.
    * В регионе 1 средний показатель запаса предсказанного сырья составляет **68.7** (ниже остальных регионов). Однако RMSE в данном регионе также мал (**0.9**). Это говорит о точности предсказаний и качестве построенной модели.
    * Средние показатели предсказанного сырья для одной скважины ниже теоретически необходимых (**111.11**)
* При расчёте показателей для 200 наилучших скважин из выборочных 500:
    * Средний запас сырья с одной скважины среди всех регионов превосходит минимально необходимый объём.
* При применении техники `Bootstrap`:
    * Оценка средней прибыли максимальна для региона 1 (**511.54 млн**.).
    * Только Регион 1 по уровню убытков (**0.80%**) не превосходит заданную границу по техническому заданию (2.5%).
    * Рекомендуем регион 1 для разработки.